In [2]:
import os
import numpy as np
from PIL import Image
import cv2
import pickle

#looks for path where current python file is saved
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
#looks for the image folder where the images are saved
image_dir = os.path.join(BASE_DIR,"img")

current_id=0

#empty dictionary
label_ids={}

#empty list
y_labels=[]
x_train=[]

face_cascade=cv2.CascadeClassifier('Cascades\haarcascade_frontalface_default.xml')
recognizer=cv2.face.LBPHFaceRecognizer_create()

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("jpg") or file.endswith("png") or file.endswith("jpeg"): 
            #path of all the jpg and png files in the img folder
            path=os.path.join(root, file)
            #grabs labels from directory
            label=os.path.basename(os.path.dirname(path))
            #print(path, label)
            
            #training labels
            if not label in label_ids:
                label_ids[label]=current_id
                current_id += 1
            id_ = label_ids[label]
            #pint(label_ids)
            
            
            pil_image=Image.open(path).convert("L") #opens the image and converts to grayscale
            resized_image=pil_image.resize((550,550), Image.ANTIALIAS) #resize the image
            image_array=np.array(resized_image,"uint8") #convert to a numpy array
            #print(image_array)
            
            #face detection inside of image
            faces=face_cascade.detectMultiScale(image_array,scaleFactor=1.5,minNeighbors=5)
            
            for (x,y,w,h) in faces:
                roi=image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
    
print(label_ids)

with open("labels.pickle",'wb') as f:
    pickle.dump(label_ids,f)
        
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")

{'Kit': 0, 'Maisie': 1, 'Rachael': 2, 'Sophie': 3}


In [3]:
print(x_train)

[array([[163, 163, 162, ..., 164, 164, 164],
       [163, 163, 162, ..., 164, 164, 164],
       [163, 163, 161, ..., 164, 164, 164],
       ...,
       [ 60,  64,  63, ...,  22,  20,  13],
       [ 61,  58,  49, ...,  21,  19,  14],
       [ 62,  54,  48, ...,  20,  20,  15]], dtype=uint8), array([[113, 113, 114, ..., 127, 125, 125],
       [114, 113, 113, ..., 126, 124, 124],
       [116, 115, 113, ..., 126, 125, 125],
       ...,
       [142, 143, 145, ..., 128, 128, 125],
       [142, 142, 145, ..., 129, 128, 125],
       [142, 144, 147, ..., 128, 128, 125]], dtype=uint8), array([[34, 35, 35, ..., 35, 35, 34],
       [34, 35, 35, ..., 35, 35, 34],
       [34, 35, 35, ..., 35, 35, 34],
       ...,
       [37, 54, 64, ...,  4,  3,  1],
       [38, 64, 75, ...,  4,  4,  2],
       [40, 70, 82, ...,  4,  5,  3]], dtype=uint8), array([[104,  95, 131, ...,  78, 168, 168],
       [118, 135, 153, ...,  68, 128, 174],
       [115, 156, 102, ..., 105,  79, 171],
       ...,
       [ 27,  26, 

In [4]:
print(y_labels)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [ ]:
#References
#[1] CodingEntrepreneurs, YouTube, 11-Apr-2018. [Online]. Available: https://www.youtube.com/watch?v=PmZ29Vta7Vc&list=WL&index=35&t=0s.
#[2] https://www.hackster.io/mjrobot/real-time-face-recognition-an-end-to-end-project-a10826